In [1]:
#13 June 2022 
#change of sensor compressor_temperature1 to compressor_differential_pressure
#setpoint rsc_rxs_tempvac_temp15k from 20 to 21k

In [2]:

import requests # Could use the builtin urllib but Requests are nicer.
from requests.exceptions import ConnectionError
import katpoint
import time
import pandas as pd
import csv
import re
import grequests 
import urlparse

%pylab inline
%autosave 60

/usr/local/lib/python2.7/dist-packages/katpoint/__init__.py:81: FutureWarning: Python 2 has reached End-of-Life, and a future version of katpoint will remove support for it. Please update your scripts to Python 3 as soon as possible.
  _warnings.warn(_PY2_WARNING, FutureWarning)


Populating the interactive namespace from numpy and matplotlib


Autosaving every 60 seconds


In [3]:
# utility function for retrieving sensor tuple form
def get_tuple(sensor_name, sensors):
    for sensor in sensors:
        if sensor[0] == sensor_name:
            return sensor

In [4]:
url="http://portal.mkat.karoo.kat.ac.za/katstore/api/query" # site for the most resent values
#url='http://kat-flap-cpt.mkat.control.kat.ac.za/katstore/samples' # flap in CT for older values

In [5]:
sensors = [
#    'rsc_rxl_state',
    ['anc_weather_temperature', 50,'gt'],
    ['rsc_rsc_he_compressor_supply_pressure', 18,'lt'],
    ['rsc_rsc_he_compressor_differential_pressure', 13,'gt'],
    ['rsc_rxl_rfe1_temperature', 26,'gt'],
    ['rsc_rxl_cryo_stepper_motor_rpm', 80,'gt'],
    ['rsc_rxl_rfe_heater1_current', 10,'lt'],
    ['rsc_rxu_rfe1_temperature', 26,'gt'],
    ['rsc_rxu_cryo_stepper_motor_rpm', 80,'gt'],
    ['rsc_rxu_rfe_heater1_current', 10,'lt'],
    ['rsc_rxs_tempvac_temp15k', 21,'gt'],
    ['rsc_rxs_cryocooler_motorspeedmeasure', 75,'gt'],
    ['rsc_rxs_tempvac_temp70k', 65,'gt']
    ]

In [6]:
aps = []
for i in range(64):
    aps.append('m'+str(i).zfill(3))

In [ ]:
now = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(time.time()))
def_start = katpoint.Timestamp(time.time()-30*24*3600)
def_end = katpoint.Timestamp(now)


all_data = {}        
for ap in aps:
    all_data[ap] = {}
    st =  time.time()
    len_rec = 0
    for i,sensor in enumerate(sensors): 
        sen = '%s_%s'%(ap,sensor[0]) if sensor[0].startswith('rsc') else '%s'%(sensor[0])  
        
        params={'sensor': sen,  # name of the sensor.
           'start_time':def_start.secs, # start time in seconds - float should work.
           'end_time': def_end.secs, # end time in seconds
           'limit':1000000,
           'include_value_time': True}
        for t in range(10):
            try:
                res = requests.get(url, params,timeout=200) # Make the call.
            except Exception as e:
                print e
                print "Retrying"
                continue
            break
        res.close()
        if  res.status_code != 200: 
            continue
        len_rec += len(res.json()['data'])
        data_list = [(r['value_time'], r['value']) for r in res.json()['data']]
        data = array(data_list)
        if len(data) !=0:
            err = any(data[:,1] >= sensor[1]) if sensor[2] == 'gt' else any(data[:,1] <= sensor[1])
            all_data[ap][sensor[0]] = {'ts':data[:,0],'data':data[:,1],'err':err}
    ed =  time.time()
    print "%s processed in %.2f seconds with %d records at %s"%(ap,ed-st,len_rec,time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(time.time())))

m000 processed in 77.24 seconds with 1721068 records at 2022-06-13 09:15:06
m001 processed in 79.22 seconds with 1779782 records at 2022-06-13 09:16:26
m002 processed in 79.32 seconds with 1795479 records at 2022-06-13 09:17:45
m003 processed in 82.08 seconds with 1870493 records at 2022-06-13 09:19:07
m004 processed in 80.48 seconds with 1801024 records at 2022-06-13 09:20:27
m005 processed in 82.92 seconds with 1892015 records at 2022-06-13 09:21:50
m006 processed in 78.19 seconds with 1772519 records at 2022-06-13 09:23:09
m007 processed in 81.42 seconds with 1852203 records at 2022-06-13 09:24:30
m008 processed in 82.45 seconds with 1904782 records at 2022-06-13 09:25:52
m009 processed in 79.10 seconds with 1808323 records at 2022-06-13 09:27:11
m010 processed in 83.92 seconds with 1911096 records at 2022-06-13 09:28:35
m011 processed in 66.01 seconds with 1444987 records at 2022-06-13 09:29:41
m012 processed in 83.18 seconds with 1899885 records at 2022-06-13 09:31:05
m013 process

In [ ]:
# Choose your xtick format string
date_fmt = '%d-%m-%y\n%H:%M:%S'

# Use a DateFormatter to set the data to the correct format.
date_formatter = DateFormatter(date_fmt)
        
for ap in aps:
    err_state = []
    for k in all_data[ap].iteritems():
        err_state.append(k[1]['err'])
    if True:
        figure(figsize=(20,10))
        for i,sensor in enumerate(sensors): 
            sen = '%s_%s'%(ap,sensor[0]) if sensor[0].startswith('rsc') else '%s'%(sensor[0])
            subplot(4,3,i+1)
            try:
                ts = all_data[ap][sensor[0]]['ts']
                data = all_data[ap][sensor[0]]['data']
            except:
                continue
            plot_date(matplotlib.dates.epoch2num(ts),data)
            title('%s'%(sen))
            ax = gca()
            axhline(sensor[1],color='r',linewidth=3)
            ax.xaxis.set_major_formatter(date_formatter)#ax.figure.autofmt_xdate()
            ax.figure.autofmt_xdate()
            grid()
    print ap      

In [ ]:

now = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(time.time()))
start = katpoint.Timestamp(time.time()-30*24*3600)
end = katpoint.Timestamp(now)


all_data = {}        
for ap in aps:
    all_data[ap] = {}
    st =  time.time()
    len_rec = 0
    len_rec = get_ap_sensors(ap,len_rec, all_data)
    ed =  time.time()
    print "%s processed in %.2f seconds with %d records at %s"%(ap,ed-st,len_rec,time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(time.time())))

